In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import json

Mechanika <b>Function Calling</b>:

- jako dodatkowy parametr zapytania przekazujemy listę funkcji (czy też narzędzi) w postaci nazwy, opisu oraz zestaw parametrów

- poza tym przesyłamy także listę wiadomości, podobnie jak w klasycznej interakcji z modelem

- model w odpowiedzi zwraca nam nazwę funkcji oraz listę wartości jej parametrów

- opcjonalnie (ale jest to zwykle potrzebne) wskazujemy, która funkcja ma być wybrana jako domyślna. Ważne: jeśli jej nie wskażemy, a model uzna, że żadna z dostępnych funkcji nie powinna zostać uruchomiona, to w 

In [2]:
queryEnrichmentSchema = {
    "name": "query_enrichment",
    "description": "Describe users query with semantic tags and classify with type",
    "parameters": {
        "type": "object",
        "properties": {
            "command": {
                "type": "boolean",
                "description": "Set to 'true' when query is direct command for AI. Set to 'false' when queries asks for saying/writing/translating/explaining something and all other."
            },
            "type": {
                "type": "string",
                "description": "memory (queries about the user and/or AI), notes|links (queries about user's notes|links). By default pick 'memory'.",
                "enum": ["memory", "notes", "links"]
            },
            "tags": {
                "type": "array",
                "description": "Multiple semantic tags/keywords that enriches query for search purposes (similar words, meanings). When query refers to the user, add 'overment' tag, and when refers to 'you' add tag 'Alice'",
                "items": {
                    "type": "string"
                }
            }
        },
        "required": [
            "type", "tags", "command"
        ]
    }
}

In [3]:
def parse_function_call(result):
    try:
        result.additional_kwargs['function_call']
        function_call = result.additional_kwargs['function_call']
        return {
            "name": function_call["name"],
            "args": json.loads(function_call["arguments"])
        }
    except:
        return None

Ogólny schemat pracy z Function Calling:

In [4]:
model = ChatOpenAI(
    model = "gpt-4-0613"
).bind(
    functions= [queryEnrichmentSchema],
    function_call= {"name": "query_enrichment"},
)

# print([queryEnrichmentSchema])
# print({"name": "query_enrichment"})

result = model.invoke([
    HumanMessage("Cześć, jestem Justynka i jutro wracam do domu :)")
])

# Analizuj wywołanie funkcji z wyniku
action = parse_function_call(result)

# Wyświetl akcję, jeśli jest dostępna
if action:
    print(action["name"], action["args"])
else:
    print("Brak wywołania funkcji w wyniku.")

query_enrichment {'command': False, 'type': 'memory', 'tags': ['Justynka', 'wracam', 'dom', 'jutro']}


Kolejnym krokiem jest faktyczne wykonanie wskazanej akcji. Np. w ten sposób, gdybyśmy mieli zdefiniowaną funckję dodawania:

In [11]:
function_name = "add"
functions_args = [5, 4]

functions = {
    "add": lambda a, b: a + b
}

if function_name in functions:
    result = functions[function_name](*functions_args)
    print(f"Wynik to {result}")
else:
    print(f"Funkcja {function_name} nie istnieje.")

Wynik to 9
